# Atmo Model Training Notebook

Train an Atmo Model using `usl_models` lib.

In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import keras
from datetime import datetime, timedelta
import os, time
from usl_models.atmo_ml.model import AtmoModel, AtmoModelParams
from usl_models.atmo_ml import dataset
from google.cloud import storage

import logging

logging.getLogger().setLevel(logging.INFO)

# climateiq-study-area-feature-chunks/NYC_Heat/NYC_summer_2000_01p
# Define bucket names and folder paths
data_bucket_name = "climateiq-study-area-feature-chunks"
label_bucket_name = "climateiq-study-area-label-chunks"
time_steps_per_day = 6
batch_size = 4

sim_dirs = [
    ('NYC_Heat_Test', [
        'NYC_summer_2000_01p',
        # 'NYC_summer_2010_99p',
        # 'NYC_summer_2015_50p',
        # 'NYC_summer_2017_25p',
        # 'NYC_summer_2018_75p'
    ]),
    ('PHX_Heat_Test', [
        # 'PHX_summer_2008_25p',
        # 'PHX_summer_2009_50p',
        # 'PHX_summer_2011_99p',
        # 'PHX_summer_2015_75p',
        # 'PHX_summer_2020_01p'
    ])
]

sim_names = []
for sim_dir, subdirs in sim_dirs:
    for subdir in subdirs:
        sim_names.append(sim_dir + '/' + subdir)

print(sim_names)
client = storage.Client(project="climateiq")
feature_bucket = client.bucket(data_bucket_name)
label_bucket = client.bucket(label_bucket_name)


2025-01-24 18:44:41.833683: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 18:44:41.883726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-24 18:44:41.883762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-24 18:44:41.884970: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 18:44:41.892910: I tensorflow/core/platform/cpu_feature_guar

['NYC_Heat_Test/NYC_summer_2000_01p']


In [2]:
inputs, label = dataset.load_day(
    date=datetime.strptime("2000-05-25", dataset.DATE_FORMAT),
    sim_name=sim_names[0],
    feature_bucket=feature_bucket,
    label_bucket=label_bucket,
)
label.shape

2025-01-24 18:44:44.536905: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37027 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


TensorShape([8, 200, 200, 5])

In [3]:
train_frac = 0.8

# Create training dataset with fused spatiotemporal data
train_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    dates=["2000-05-25"],
).batch(batch_size=1)
for inputs, labels in train_ds:
    print(labels.shape)
    break

INFO:root:Total simulation days before filtering: 1
INFO:root:Selected 1/1 days (100.00%) based on hash range (0.0, 1.0).


sim_name_dates [('NYC_Heat_Test/NYC_summer_2000_01p', '2000-05-25')]


INFO:root:Total generated samples: 1


(1, 8, 200, 200, 5)


In [11]:
# Create training dataset with fused spatiotemporal data
train_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    dates=["2000-05-25"],
).batch(batch_size=batch_size)

# Create validation dataset with fused spatiotemporal data
val_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    dates=["2000-05-25"],
).batch(batch_size=batch_size)

INFO:root:Total simulation days before filtering: 1
INFO:root:Selected 1/1 days (100.00%) based on hash range (0.0, 1.0).
INFO:root:Total simulation days before filtering: 1
INFO:root:Selected 1/1 days (100.00%) based on hash range (0.0, 1.0).


sim_name_dates [('NYC_Heat_Test/NYC_summer_2000_01p', '2000-05-25')]
sim_name_dates [('NYC_Heat_Test/NYC_summer_2000_01p', '2000-05-25')]


In [ ]:
# Initialize the Atmo Model
model_params = AtmoModelParams()
model = AtmoModel(model_params)

In [ ]:
# Create a unique log directory by appending the current timestamp
log_dir = os.path.join("./logs", "run_" + time.strftime("%Y%m%d-%H%M%S"))

# Set up TensorBoard callback
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
print(log_dir)

./logs/run_20250124-184506


In [ ]:
# Train the model
model.fit(train_ds, val_ds, epochs=150, callbacks=[tb_callback])

Epoch 1/150
Output shape: TensorShape([None, 8, 50, 50, 256])


2025-01-24 18:46:21.277779: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: load_day() got an unexpected keyword argument 'storage_client'
Traceback (most recent call last):

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/josiahkp/climateiq-cnn/usl_models/usl_models/atmo_ml/dataset.py", line 276, in data_generator
    load_result = load_day(
                  ^^^^^^^^^

TypeError: load_day() got an unexpected keyword a

InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  TypeError: load_day() got an unexpected keyword argument 'storage_client'
Traceback (most recent call last):

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/josiahkp/climateiq-cnn/us

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get predictions from the validation set
predictions = model._model.predict(val_ds)  # Use the underlying Keras model

# Assuming the structure of val_ds returns (input_data, ground_truth)
for input_data, ground_truth in val_ds.take(1):  # Taking just one batch from val_ds
    # Get predicted labels
    predicted_labels = model._model.predict(input_data)
    
    # Compute shared vmin and vmax for consistent color range
    vmin = min(
        np.min(ground_truth[:, 0, :, :, 0]), 
        np.min(predicted_labels[:, 0, :, :, 0])
    )
    vmax = max(
        np.max(ground_truth[:, 0, :, :, 0]), 
        np.max(predicted_labels[:, 0, :, :, 0])
    )
    
    # Visualize the first sample
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), dpi=150)  # Higher DPI for quality
    
    # Ground Truth Visualization
    img1 = axes[0].imshow(
        ground_truth[0, 0, :, :, 0], 
        cmap='viridis', 
        vmin=90, 
        vmax=100
    )
    axes[0].set_title('Ground Truth')
    plt.colorbar(img1, ax=axes[0], fraction=0.046, pad=0.04)

    # Prediction Visualization
    img2 = axes[1].imshow(
        predicted_labels[0, 0, :, :, 0], 
        cmap='viridis', 
        vmin=vmin, 
        vmax=vmax
    )
    axes[1].set_title('Predicted Labels')
    plt.colorbar(img2, ax=axes[1], fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()
    break  # Break after visualizing one batch

In [ ]:
inputs, labels = next(iter(train_ds))
{key: tensor.shape for key, tensor in inputs.items()}

In [ ]:
model._model.summary()

In [ ]:
# Test calling the model on some training data
inputs, labels = next(iter(train_ds))
prediction = model.call(inputs)
print("Prediction shape:", prediction.shape)